# 理解javascript函数调用和“this”

Over the years, I've seen a lot of confusion about JavaScript function invocation. In particular, a lot of people have complained that the semantics of this in function invocations is confusing.

多年以来，我看到大量关于JavaScript函数调用的困惑。尤其，许多人抱怨函数调用中“this”的语意是混乱的。

In my opinion, a lot of this confusion is cleared up by understanding the core function invocation primitive, and then looking at all other ways of invoking a function as sugar on top of that primitive. In fact, this is exactly how the ECMAScript spec thinks about it. In some areas, this post is a simplification of the spec, but the basic idea is the same.

在我看来，大量这样的混乱可以通过理解核心函数调用原语被清理，然后再看所有其他在原语之上进行包装的调用函数的方法。实际上，这正好是ECMAScript规格对这个问题的考虑。在某些领域，这个是一个规格的简化，但基本思想是一样的。

## The Core Primitive 核心原语

First, let's look at the core function invocation primitive, a Function's call method[1]. The call method is relatively straight forward.

首先，我们来看核心函数调用原语，一个函数的`call`方法[1]。这个调用方法比较简单直接。

1. Make an argument list (argList) out of parameters 1 through the end
2. The first parameter is thisValue
3. Invoke the function with this set to thisValue and the argList as its argument list


1. 构造参数列表（argList）从参数1到最后
2. 第一个参数是 thisValue
3. 把 this 赋值给 thisValue 并用 argList 作为参数列表调用函数

例如：

In [ ]:
function hello(thing) {  
  console.log(this + " says hello " + thing);  
}

hello.call("Yehuda", "world") //=> Yehuda says hello world 

As you can see, we invoked the hello method with this set to "Yehuda" and a single argument "world". This is the core primitive of JavaScript function invocation. You can think of all other function calls as desugaring to this primitive. (to "desugar" is to take a convenient syntax and describe it in terms of a more basic core primitive).

正如你看到的，我们通过把`this`赋值给 "Yehuda"和一个单一参数"world"来调用 hello 方法。这就是 javascript 函数调用核心原语。你能想象所有其他的函数调用都是对这个原语包装。(包装是使用一个便利语法按照更基本的核心原语描述它)

_[1] In the ES5 spec, the call method is described in terms of another, more low level primitive, but it’s a very thin wrapper on top of that primitive, so I’m simplifying a bit here. See the end of this post for more information._

_[1] 在 ES5 规范中，`call`方法是根据另一个较低级别的原语来描述的，但它在该原语之上是一个非常简单的包装器，所以我在这里简化了一点。 有关更多信息，请参阅本文的末尾。_

## Simple Function Invocation 简单函数调用

Obviously, invoking functions with call all the time would be pretty annoying. JavaScript allows us to invoke functions directly using the parens syntax (hello("world"). When we do that, the invocation desugars:

很明显，任何时候使用 `call` 调用函数都是相当的烦人的。Javascript允许我们使用括弧直接调用函数（`hello("world")`）。我们这样做的时候，调用包装为：

In [ ]:
function hello(thing) {  
  console.log("Hello " + thing);  
}  
   
// this:  
hello("world")  
   
// desugars to:  
hello.call(window, "world");  

This behavior has changed in ECMAScript 5 only when using strict mode[2]:

这个行为在 ECMAScript 5 中只有当使用`strict`模式时才会发生改变：

In [ ]:
// this:  
hello("world") 

// desugars to:  
hello.call(undefined, "world");  

The short version is: a function invocation like fn(...args) is the same as fn.call(window [ES5-strict: undefined], ...args).

短版本：函数调用`fn(...args)`和`fn.call(window [ES5-strict: undefined], ...args)`等同。

Note that this is also true about functions declared inline: (function() {})() is the same as (function() {}).call(window [ES5-strict: undefined).

需要注意的是，函数内联声明也是正确的：`(function() {})()`和`(function(){}).call(window [ES5-strict: undefined)`等同。

[2] Actually, I lied a bit. The ECMAScript 5 spec says that undefined is (almost) always passed, but that the function being called should change its thisValue to the global object when not in strict mode. This allows strict mode callers to avoid breaking existing non-strict-mode libraries.

[2]实际上，我说了点谎。ECMAScript 5 规范说一般（大多情况）传递的是`undefined`，但被调用的函数在非严格模式时应该改变它的 thisValue 为全局对象。这允许 `strict` 模式调用者避免破坏现存的非严格模式库。

## 成员函数

The next very common way to invoke a method is as a member of an object `(person.hello())`. In this case, the invocation desugars:

下一个非常常见的方法是调用作为对象的成员方法`(person.hello())`。在这种情况下，调用包装为：

In [ ]:
var person = {  
  name: "Brendan Eich",  
  hello: function(thing) {  
    console.log(this + " says hello " + thing);  
  }  
}  
   
// this:  
person.hello("world")  
   
// desugars to this:  
person.hello.call(person, "world"); 

Note that it doesn't matter how the hello method becomes attached to the object in this form. Remember that we previously defined hello as a standalone function. Let's see what happens if we attach is to the object dynamically:

注意，`hello` 方法如何以这种方式附加到对象上的并不重要。 记住我们之前作为独立函数定义的 `hello` 。我们来看看如果动态的附加到对象上发生了什么：

In [ ]:
function hello(thing) {  
  console.log(this + " says hello " + thing);  
}  
   
person = { name: "Brendan Eich" }  
person.hello = hello;  
   
person.hello("world") // still desugars to person.hello.call(person, "world")  
   
hello("world") // "[object DOMWindow]world" 

Notice that the function doesn't have a persistent notion of its 'this'. It is always set at call time based upon the way it was invoked by its caller.

注意这个函数没有确定的“this”概念。它总是在调用时基于它的调用者的调用方式设置。

## 使用 Function.prototype.bind

Because it can sometimes be convenient to have a reference to a function with a persistent this value, people have historically used a simple closure trick to convert a function into one with an unchanging this:

因为有时有一个确定 `this` 值的函数引用会方便一些，人们使用一个简单的封闭把戏来转换一个函数为一个不变的 `this` :

In [ ]:
var person = {  
  name: "Brendan Eich",  
  hello: function(thing) {  
    console.log(this.name + " says hello " + thing);  
  }  
}  
   
var boundHello = function(thing) { return person.hello.call(person, thing); }  
   
boundHello("world");  

Even though our `boundHello` call still desugars to `boundHello.call(window, "world")`, we turn right around and use our primitive call method to change the `this` value back to what we want it to be.

尽管通过我们的 `boundHello` 调用仍然解释为 `boundHello.call(window,"world")`，我们转了一圈然然后使用我们原语调用方法来修改 `this` 值为我们期望的值。

We can make this trick general-purpose with a few tweaks:

我们可以进行一些调整达到这个通用的目的：

In [ ]:
var bind = function(func, thisValue) {  
  return function() {  
    return func.apply(thisValue, arguments);  
  }  
}  
   
var boundHello = bind(person.hello, person);  
boundHello("world") // "Brendan Eich says hello world"  

In order to understand this, you just need two more pieces of information. First, `arguments` is an Array-like object that represents all of the `arguments` passed into a function. Second, the `apply` method works exactly like the `call` primitive, except that it takes an Array-like object instead of listing the arguments out one at a time.

为了理解这一点，你只需要再多了解两条信息。首先，`arguments` 是一个代表所有传递到函数的参数的类数组对象。其次，`apply` 方法和 `call` 原语工作原理完全一样，除了它带了一个类似数组的对象代替参数列表。

Our `bind` method simply returns a new function. When it is invoked, our new function simply invokes the original function that was passed in, setting the original value as `this`. It also passes through the arguments.

我们的 `bind` 方法简单的返回一个新的函数。当它被调用时，我们的新函数简单的调用了传入的原始函数，设置原始值为 `this`。它也通过参数传递。

Because this was a somewhat common idiom, ES5 introduced a new method `bind` on all `Function` objects that implements this behavior:

因为这是一个比较常见的习语，ES5为所有的 `Function` 对象引入一个新的 `bind` 方法，它实现下面的行为：

In [ ]:
var boundHello = person.hello.bind(person);  
boundHello("world") // "Brendan Eich says hello world"  

This is most useful when you need a raw function to pass as a callback:

当你需要一个原始函数作为回调传递时这更有用：

In [ ]:
var person = {  
  name: "Alex Russell",  
  hello: function() { console.log(this.name + " says hello world"); }  
}  
   
$("#some-div").click(person.hello.bind(person));  
   
// when the div is clicked, "Alex Russell says hello world" is printed  

This is, of course, somewhat clunky, and TC39 (the committee that works on the next version(s) of ECMAScript) continues to work on a more elegant, still-backwards-compatible solution.

当然，这有点笨拙，并且TC39（委员会正在制定的下一个ECMAScript版本）继续致力于更优雅的，向后兼容的方案。

## 在jQuery中

Because `jQuery` makes such heavy use of anonymous callback functions, it uses the `call` method internally to set the this value of those callbacks to a more useful value. For instance, instead of receiving window as this in all event handlers (as you would without special intervention), jQuery invokes call on the callback with the element that set up the event handler as its first parameter.

因为 `jQuery` 大量使用匿名回调函数，它内部使用 `call` 方法设置那些回调的 `this` 值为更有用的值。例如， 在所有的事件处理中替代接收 `window` 作为 `this`（如你没有特别的处理），`jQuery` 在回调中使用建立事件处理器的元素作为它的第一个参数调用 `call` 。

This is extremely useful, because the default value of this in anonymous callbacks is not particularly useful, but it can give beginners to JavaScript the impression that this is, in general a strange, often mutated concept that is hard to reason about.

这非常有用，因为匿名调用中 `this` 的默认值不是特别的有用，但它可以给JavaScipt初学者的印象，this 通常是奇怪的，常常改变，很难确定的概念。

If you understand the basic rules for converting a sugary function call into a desugared func.call(thisValue, ...args), you should be able to navigate the not so treacherous waters of the JavaScript this value.

如果你理解了基本规则，转换一个包装的函数调用为一个非包装的`func.call(thisValue,...args)`，你应该能导航不那么变化莫测的Javascript `this` 值水域。


## 附录：I Cheated 我行骗了

In several places, I simplified the reality a bit from the exact wording of the specification. Probably the most important cheat is the way I called func.call a "primitive". In reality, the spec has a primitive (internally referred to as `[[Call]]`) that both `func.call` and `[obj.]func()` use.

在一些地方，我从规格精确的语法简化了现实一点。可能最主要的欺骗是我称 `func.call` 为“原语”的习惯。实际上，该规格有一个原语（内部引用为 `[[Call]]`）被 `func.call` 和`[obj.]func()` 使用。

However, take a look at the definition of `func.call`:

不管怎样，一起来看看 `func.call` 定义：

1. If `IsCallable(func)` is `false`, then throw a `TypeError` exception.
2. Let `argList` be an empty List.
3. If this method was called with more than one argument then in left to right order starting with arg1 append each argument as the last element of argList
4. Return the result of calling the `[[Call]]` internal method of func, providing thisArg as the this value and argList as the list of arguments.


1. 如果 `IsCallable(func)` 是 `false`，则抛出一个 `TypeError` 异常。
2. 置 `argList` 为空列表。
3. 如果方法被调用时使用超过一个参数，那么以从左到右顺序从 `arg1` 开始追加每个参数作为`argList` 最后的元素。
4. 返回调用func的 `[[Call]]` 内部方法的结果，提供 `thisArg` 作为 `this` 值，和 `argList` 作为参数列表。

As you can see, this definition is essentially a very simple JavaScript language binding to the primitive `[[Call]]` operation.

正如你看到的，这个定义是本质非常简单的对原语 `[[Call]]` 操作的 JavaScript 语言绑定。

If you look at the definition of invoking a function, the first seven steps set up thisValue and argList, and the last step is: "Return the result of calling the [[Call]] internal method on func, providing thisValue as the this value and providing the list argList as the argument values."

如果你看一下调用函数的定义，七步的第一步是建立 `thisValue` 和 `argList`，最后一步是：“返回在func中调用 `[[Call]]` 内部方法的结果，提供 `thisValue` 作为 `this` 值并提供参数列表作为 `argument` 的值。”

It's essentially identical wording, once the argList and thisValue have been determined.

它的本质是相同的语法，一旦 `argList` 和 `thisValue` 被确定。

I cheated a bit in calling `call` a primitive, but the meaning is essentially the same as had I pulled out the spec at the beginning of this article and quoted chapter and verse.

我撒了一点谎，在把 `call` 叫做原语，但它的意思的本质是相同的，所以在本文的开始和引用的章节与段落我拉出规格。

There are also some additional cases (most notably involving with) that I didn't cover here.

这也有一些附加的情况（尤其包括）在这里没有提到情况。


[原文地址](http://yehudakatz.com/2011/08/11/understanding-javascript-function-invocation-and-this/)

